In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# putting mutiple websites to scrape
# start using the url to get into the scraping process
#change the website details as I have changed to women's team roster
urls = [
    'https://bmccathletics.com/sports/womens-volleyball/roster?view=2',
    'https://yorkathletics.com/sports/womens-volleyball/roster',
    'https://hostosathletics.com/sports/womens-volleyball/roster',
    'https://bronxbroncos.com/sports/womens-volleyball/roster/2021',
    'https://queensknights.com/sports/womens-volleyball/roster',
    'https://augustajags.com/sports/wvball/roster',
    'https://flaglerathletics.com/sports/womens-volleyball/roster',
    'https://pacersports.com/sports/womens-volleyball/roster',
    'https://www.golhu.com/sports/womens-volleyball/roster'
]

# defining the data/ it is important to create the data frame using pandas in the future
data = []

# Iterate over the URLs
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # use soup find all to scrape the whole table
    players = soup.find_all('tr')
    
    for player in players:
        # use td and class to get into details with names and heights 
        player_names = player.find('td', class_='sidearm-table-player-name')
        player_heights = player.find('td', class_='height')
        
        # If both name and height elements are found, extract the text
        if player_names and player_heights:
            name = player_names.text.strip()
            height = player_heights.text.strip()
            data.append({'Name': name  , 'Height': height })

# Convert data into a DataFrame
df = pd.DataFrame(data)
df['Height'] = df['Height'].str.replace('-', "'") + '"' # Convert to a more uniform format, e.g., 6'5"

# Function to convert height from feet-inches to inches
def height_to_inches(height):
    try:
        feet, inches = height.split("'")
        inches = inches.replace('"', '').strip()
        return int(feet) * 12 + int(inches)
    except ValueError as e: # to see if there is some errors in the converting height to inches
        print(f"Error converting height '{height}' to inches: {e}")
        return None


# Apply the conversion to the DataFrame
df['Height_in_inches'] = df['Height'].apply(height_to_inches)

# started changing to the table and convert into the csv
# Calculate the average height in inches and sort data for analysis
average_height_in_inches = df['Height_in_inches'].mean()
df_sorted = df.sort_values(by='Height_in_inches', ascending=True)
tallest_players = df_sorted.tail(5)
shortest_players = df_sorted.head(5)

# Save to CSV
df.to_csv('women_volleyball.csv', index=False) # convert the dataframe to csv file

# Output results
print(f"The average height of the players across all teams is {average_height_in_inches:.2f} inches.")
print("\nTallest Players:\n", tallest_players[['Name', 'Height']])
print("\nShortest Players:\n", shortest_players[['Name', 'Height']])
print("saved to file") # to make sure that is saved to the file

Error converting height ''"' to inches: invalid literal for int() with base 10: ''
The average height of the players across all teams is 67.46 inches.

Tallest Players:
                      Name Height
68           Madelyn Eden   6'2"
63         Sophia Kruczko   6'3"
78             Molly Boyd   6'3"
114          Alyssa Daley   6'3"
3    Marie Claire Hurtado     '"

Shortest Players:
                  Name Height
26      Samara Correa   5'0"
1    Yisneily Morales   5'1"
4     Garyana Altidor   5'1"
101  Izabella De Lima   5'1"
17      Jasmine  Vega   5'1"
saved to file
